# Discovering Sentinel-2 products

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import visualize_S2_products 

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import json

import geopandas as gpd
import pandas as pd
pd.set_option('display.max_colwidth', None)
import folium
import folium.plugins
import logging
import numpy as np

import shapely
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [ ]:
# load a geometry
file_aoi = "/data/aoi_Maroc.geojson"
# file_aoi = "/data/aoi_bretagne.geojson"
gj_aoi = read_geojson(file_aoi)
aoi = geojson_to_wkt(gj_aoi["features"][0])
shp = gpd.read_file(file_aoi).geometry[0]

# open credentials
with open("/data/creds.json") as f:
    creds = json.load(f)

# search for products intersecting geometry
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    platformname="Sentinel-2",
    date=("20230901", "20230920"),
    producttype="S2MSI1C",
    #  orbitdirection='ASCENDING',
    limit=50,
)

# make a geodataframe for visualization
gdf = api.to_geodataframe(products).reset_index(names='id')
dates = np.unique(gdf.beginposition.astype(str)).tolist()
print(f"Unique dates:\n{"\n".join(dates)}")

In [ ]:
visualize_S2_products(gdf, dates[7], gj_aoi)

## Download

In [ ]:
# fill indices of products to download
indices_to_download = [1, 5]
ids = gdf['id'][indices_to_download]
# dl = api.download_all_quicklooks(ids, directory_path='/data/S1/')
dl = api.download_all(ids, directory_path='/data/S1/')

In [ ]:
from localtileserver import TileClient, get_folium_tile_layer
sel = gdf[gdf.beginposition == dates[1]]
dl = api.download_all_quicklooks(sel.id, directory_path='/data/tmp/')
# client = TileClient(file_in)
# t = get_folium_tile_layer(client, vmin=0.0, vmax=1.0)

# m = folium.Map(location=client.center(), zoom_start=client.default_zoom)
# t.add_to(m)

In [ ]:
# dev: geocode & merge quicklooks
from shapely.wkt import loads
import rasterio
from rasterio.merge import merge
from rasterio.transform import from_gcps, GroundControlPoint

opened_images = []

for _, item in dl[0].items():
    footprint = loads(item['footprint'])
    image_path = item['path']
    with rasterio.open(image_path) as src:
        h, w = src.height, src.width
    #     src.transform = rasterio.transform.from_origin(
    #         src.bounds.left, src.bounds.top, pixel_size, pixel_size
    #     )
    #     opened_images.append(src)

# Mosaic the opened images
# mosaic, out_transform = merge(opened_images)

# with rasterio.open('mosaic_geocoded_quicklooks.tif', 'w', driver='GTiff', width=mosaic.shape[2], height=mosaic.shape[1], count=mosaic.shape[0], dtype=mosaic.dtype, crs=src.crs, transform=out_transform) as dst:
#     dst.write(mosaic)

In [ ]:
list(footprint.exterior.coords)

In [ ]:
footprint

In [ ]:
footprint.bounds

In [ ]:
api.query?